In [1]:
import wrds
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
load_dotenv(verbose=True)


True

In [2]:
crspRaw = pd.read_csv('./CRSP2020Through2022Daily.csv')
crspRaw.columns = crspRaw.columns.str.lower()

C:\Users\joshu\AppData\Local\Temp\ipykernel_1480\952844144.py:1: DtypeWarning: Columns (5,9,18,49,57) have mixed types. Specify dtype option on import or set low_memory=False.
  crspRaw = pd.read_csv('./CRSP2020Through2022Daily.csv')


In [5]:
crspData = crspRaw.filter(['date', 'shrcd', 'exchcd', 'siccd', 'permno', 'permco', 'ret', 'shrout', 'prc', 'dlret'])
crspData['date'] = pd.to_datetime(crspData['date'],format = '%Y%m%d', errors='coerce')   # Date of data
crspData['shrcd'] = pd.to_numeric(crspData['shrcd'], errors='coerce')  # Share code
crspData['exchcd'] = pd.to_numeric(crspData['exchcd'], errors='coerce') # Exchange code
crspData['permno'] = pd.to_numeric(crspData['permno'], errors='coerce') # Security code
crspData['permco'] = pd.to_numeric(crspData['permco'], errors='coerce') # Company code
crspData['shrout'] = pd.to_numeric(crspData['shrout'], errors='coerce') # Shares outstanding (thousands)
crspData['prc'] = pd.to_numeric(crspData['prc'], errors='coerce')       # Last traded price in a month
crspData['ret'] = pd.to_numeric(crspData['ret'], errors='coerce')       # Holding return

In [6]:
crspData

,date,shrcd,exchcd,siccd,permno,permco,ret,shrout,prc,dlret
0,2020-01-02,11.0,3.0,2052,10026,7976,-0.014056,18900.0,181.67999,NaN
1,2020-01-03,11.0,3.0,2052,10026,7976,0.017779,18900.0,184.91000,NaN
2,2020-01-06,11.0,3.0,2052,10026,7976,0.000865,18900.0,185.07001,NaN
3,2020-01-07,11.0,3.0,2052,10026,7976,-0.011023,18900.0,183.03000,NaN
4,2020-01-08,11.0,3.0,2052,10026,7976,-0.005409,18900.0,182.03999,NaN
...,...,...,...,...,...,...,...,...,...,...
4715937,2022-03-25,11.0,3.0,9999,93436,53453,-0.003235,1033534.0,1010.64001,NaN
4715938,2022-03-28,11.0,3.0,9999,93436,53453,0.080345,1033534.0,1091.83997,NaN
4715939,2022-03-29,11.0,3.0,9999,93436,53453,0.007080,1033534.0,1099.56995,NaN
4715940,2022-03-30,11.0,3.0,9999,93436,53453,-0.005075,1033534.0,1093.98999,NaN


In [9]:
crspData.describe()

,date,shrcd,exchcd,permno,permco,ret,shrout,prc,date_year,me,log_me,size_rank,excess_ret
count,469544,469544.000000,469544.000000,469544.000000,469544.000000,469544.000000,469544.000000,469544.000000,469544.000000,469544.000000,469544.000000,469544.000000,469544.000000
mean,2021-03-05 14:20:57.206140672,56.360390,3.647328,31006.857080,50303.420163,-0.000395,3354.269121,21.458868,2020.711739,0.011744,-4.736699,0.050033,-0.030395
min,2020-01-02 00:00:00,11.000000,1.000000,10044.000000,357.000000,-0.812233,5.000000,0.067650,2020.000000,0.000040,-10.126631,0.000106,-0.842233
25%,2020-08-12 00:00:00,31.000000,3.000000,17067.000000,51652.000000,-0.009772,200.000000,5.110000,2020.000000,0.005285,-5.242883,0.025046,-0.039772
50%,2021-03-18 00:00:00,73.000000,4.000000,19097.000000,54276.000000,0.000000,500.000000,23.034150,2021.000000,0.010449,-4.561292,0.050033,-0.030000
75%,2021-09-30 00:00:00,73.000000,4.000000,21655.000000,55978.000000,0.008713,2303.250000,28.919525,2021.000000,0.017254,-4.059740,0.075020,-0.021287
max,2022-03-31 00:00:00,74.000000,5.000000,93367.000000,59243.000000,8.523355,234002.000000,157.616200,2022.000000,0.033488,-3.396581,0.100000,8.493355
std,NaN,26.158796,0.930962,27205.383284,11851.342586,0.046852,8261.953556,17.048902,0.673191,0.007623,0.875845,0.028852,0.046852


In [7]:
# Filter based on shrcd 
# First number (1) is the Ordinary Common Share Code
# 0	Securities which have not been further defined.
# 1	Securities which need not be further defined.
# 2	Companies incorporated outside the US   
# 3	Americus Trust Components (Primes and Scores).
# 4	Closed-end funds.
crspData[crspData['shrcd'].isin([10, 11, 12, 13, 14])]

# Filter based on exchcd
# 1	New York Stock Exchange
# 2	American Stock Exchange
# 3	The Nasdaq Stock Market(SM)
crspData[crspData['exchcd'].isin([1,2,3,31,32,33])]

# Handle missing values in 'prc' and 'shrout'
# prc is closing price avg of bid and ask
# shrout is shares outstanding
crspData['prc'] = crspData['prc'].abs()
crspData['shrout'] = crspData['shrout'].abs()

# Drop missing rows
crspData.dropna(subset=['ret'], inplace=True)

# Remove duplicates
crspData.drop_duplicates(inplace=True)

# Add a new column 'date_year' with the year extracted from 'date'
crspData['date_year'] = crspData['date'].dt.year

# Filter data from the year 2020 onwards
crspData = crspData[crspData['date_year'] >= 2020]

# Calculate market equity
crspData['me'] = crspData['shrout'] * crspData['prc'] / 1000000  # in millions


# Adjust delisting returns
crspData['ret_adj'] = crspData['ret']
for i in crspData.index:
    if pd.isnull(crspData.loc[i, 'ret']) or crspData.loc[i, 'ret'] == -99:
        crspData.loc[i, 'ret_adj'] = crspData.loc[i, 'ret']
        if pd.isnull(crspData.loc[i, 'prc']):
            crspData.loc[i, 'ret_adj'] = crspData.loc[i, 'dlret']
        else:
            if crspData.loc[i, 'dlret'] == -1:
                crspData.loc[i, 'ret_adj'] = -1
            else:
                crspData.loc[i, 'ret_adj'] = (1 + crspData.loc[i, 'dlret']) * (1 + crspData.loc[i, 'ret']) - 1
    else:
        crspData.loc[i, 'ret_adj'] = -1

# Apply log transformation to market equity
crspData['log_me'] = np.log(crspData['me'])

# Handle missing values in 'me'
crspData.dropna(subset=['me'], inplace=True)

# Add a new column 'size_rank' based on the log of market equity
crspData['size_rank'] = crspData.groupby('date')['log_me'].rank(pct=True)

# Filter stocks in the bottom decile by size
small_size_threshold = 0.1
crspData = crspData[crspData['size_rank'] <= small_size_threshold]

# Calculate the excess returns
crspData['excess_ret'] = crspData['ret'] - 0.0003




In [ ]:
# Calculate market value
crspData['me'] = np.abs(crspData['prc'] * crspData['shrout'] / 1000)

# Add year and month columns
crspData['year'] = pd.to_datetime(crspData['date']).dt.year
crspData['month'] = pd.to_datetime(crspData['date']).dt.month

# Pivot balance
pivot_balance = crspData.sort_values(by=['siccd', 'date'])

# Get deciles based on market value at the start of year
pivot_balance = pivot_balance.groupby('year').apply(lambda x: x.set_index('siccd').first())

pivot_balance['decile'] = pivot_balance.groupby('year')['me'].transform(lambda x: pd.qcut(x, 10, labels=False))

pivot_balance['decile'] = pivot_balance['decile'] + 1

# Drop missing values
pivot_balance = crspData.dropna()

# Save the result
pivot_balance.drop(['index'], inplace=True, axis=1)

print(pivot_balance)


In [31]:
crspData.describe()

,date,shrcd,exchcd,permno,permco,ret,shrout,prc,date_year,me,log_me,size_rank,excess_ret
count,469544,469544.000000,469544.000000,469544.000000,469544.000000,469544.000000,469544.000000,469544.000000,469544.000000,469544.000000,469544.000000,469544.000000,469544.000000
mean,2021-03-05 14:20:57.206140672,56.360390,3.647328,31006.857080,50303.420163,-0.000395,3354.269121,21.458868,2020.711739,11.743826,2.171057,0.050033,-0.030395
min,2020-01-02 00:00:00,11.000000,1.000000,10044.000000,357.000000,-0.812233,5.000000,0.067650,2020.000000,0.040000,-3.218876,0.000106,-0.842233
25%,2020-08-12 00:00:00,31.000000,3.000000,17067.000000,51652.000000,-0.009772,200.000000,5.110000,2020.000000,5.285000,1.664873,0.025046,-0.039772
50%,2021-03-18 00:00:00,73.000000,4.000000,19097.000000,54276.000000,0.000000,500.000000,23.034150,2021.000000,10.448550,2.346463,0.050033,-0.030000
75%,2021-09-30 00:00:00,73.000000,4.000000,21655.000000,55978.000000,0.008713,2303.250000,28.919525,2021.000000,17.253500,2.848015,0.075020,-0.021287
max,2022-03-31 00:00:00,74.000000,5.000000,93367.000000,59243.000000,8.523355,234002.000000,157.616200,2022.000000,33.487560,3.511174,0.100000,8.493355
std,NaN,26.158796,0.930962,27205.383284,11851.342586,0.046852,8261.953556,17.048902,0.673191,7.623473,0.875845,0.028852,0.046852


In [34]:
crspData[crspData['excess_ret'] > 2]

,date,shrcd,exchcd,siccd,permno,permco,ret,shrout,prc,date_year,me,log_me,size_rank,excess_ret
1228167,2020-06-09,31.0,3.0,9999,16787,55989,8.523355,599.0,20.00,2020,11.98000,2.483239,0.065028,8.493355
1234923,2020-03-31,11.0,3.0,9999,16802,55993,3.514563,1171.0,9.30,2020,10.89030,2.387872,0.079682,3.484563
1401842,2020-08-05,31.0,3.0,9999,17311,56264,3.515585,53.0,26.80,2020,1.42040,0.350939,0.003012,3.485585
2285402,2021-06-10,31.0,3.0,9999,19839,57353,3.015296,575.0,21.00,2021,12.07500,2.491137,0.042784,2.985296
3085640,2020-01-16,11.0,2.0,2711,75261,21174,2.418414,5502.0,1.50,2020,8.25300,2.110577,0.051279,2.388414
3173597,2020-06-16,11.0,2.0,4833,76963,11101,2.600000,2937.0,4.50,2020,13.21650,2.581466,0.070627,2.570000
3603142,2020-01-06,11.0,3.0,3060,85540,15800,2.103007,3836.0,2.60,2020,9.97360,2.299942,0.063291,2.073007
3620632,2020-12-28,11.0,2.0,7359,85757,34622,4.148508,1546.0,13.85,2020,21.41210,3.063956,0.087439,4.118508
3701346,2020-06-15,11.0,3.0,4832,86916,16512,2.554348,1582.0,6.54,2020,10.34628,2.336627,0.057154,2.524348


In [32]:
# Save the cleaned data
crspData.to_csv('cleaned_crsp_data.csv', index=False)